# validate results

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
import os
import argparse
from datetime import datetime

import numpy as np
import pandas as pd
from numpy.random import default_rng

from activitysim.cli import run
from activitysim.core import inject
from activitysim.core import tracing
from activitysim.core import config
from activitysim.core import pipeline
from activitysim.core import chunk
from activitysim.core import simulate
from activitysim.core import logit
from activitysim.abm.models.util import estimation
from activitysim.core import expressions
from activitysim.core.util import assign_in_place

In [4]:
pd.set_option("max_columns", 500)

In [5]:
root_dir = "/mnt/c/Users/jan.zill/code/activitysim"
example_dir = os.path.join(root_dir, "test_example_mtc")

In [6]:
os.chdir(example_dir)

In [7]:
parser = argparse.ArgumentParser()
run.add_run_args(parser)
# args = parser.parse_args()
# parser.parse_args(['--sum', '7', '-1', '42'])
args = parser.parse_args(['-c', 'configs', '-o', 'output', '-d', 'data'])
#run.run(args)  # 2mins full example run
if not inject.is_injectable('preload_injectables'):
    from activitysim import abm  # register abm steps and other abm-specific injectables
run.handle_standard_args(args)  # possibly update injectables

In [8]:
from fru_utils import run_trip_mode_choice
trips_df_ru = run_trip_mode_choice()
trips_df = run_trip_mode_choice(choose_individual_max_utility=False)

register joint_tour_participants: no rows with household_id in [982875].
estimation bundle trip_mode_choice not in settings file estimation.yaml


Running with %d trips 482
trip_mode_choice tour_type 'atwork' (27 trips)
trip_mode_choice tour_type 'eatout' (33 trips)
trip_mode_choice tour_type 'escort' (6 trips)
trip_mode_choice tour_type 'othdiscr' (43 trips)
trip_mode_choice tour_type 'othmaint' (46 trips)
trip_mode_choice tour_type 'school' (37 trips)
trip_mode_choice tour_type 'shopping' (77 trips)
trip_mode_choice tour_type 'social' (19 trips)
trip_mode_choice tour_type 'univ' (26 trips)
trip_mode_choice tour_type 'work' (168 trips)


register joint_tour_participants: no rows with household_id in [982875].


Done
Running with %d trips 482


estimation bundle trip_mode_choice not in settings file estimation.yaml


trip_mode_choice tour_type 'atwork' (27 trips)
trip_mode_choice tour_type 'eatout' (33 trips)
trip_mode_choice tour_type 'escort' (6 trips)
trip_mode_choice tour_type 'othdiscr' (43 trips)
trip_mode_choice tour_type 'othmaint' (46 trips)
trip_mode_choice tour_type 'school' (37 trips)
trip_mode_choice tour_type 'shopping' (77 trips)
trip_mode_choice tour_type 'social' (19 trips)
trip_mode_choice tour_type 'univ' (26 trips)
trip_mode_choice tour_type 'work' (168 trips)
Done


In [9]:
c_ = ["trip_mode", "mode_choice_logsum"]
trips = trips_df_ru[c_].merge(trips_df[c_], left_index=True, right_index=True, how="outer", suffixes=["_fru", "_asim"])
pd.merge(trips_df_ru.trip_mode.value_counts(), trips_df.trip_mode.value_counts(), left_index=True, right_index=True,
         suffixes=["_fru", "_asim"])

,trip_mode_fru,trip_mode_asim
WALK,299,304
WALK_LOC,92,85
WALK_LRF,61,53
BIKE,17,17
DRIVEALONEFREE,6,4
SHARED2FREE,3,4
TNC_SINGLE,2,4
WALK_HVY,2,4


In [10]:
if np.allclose(trips.mode_choice_logsum_fru, trips.mode_choice_logsum_asim):
    print("Logsums agree")
else:
    ((trips.mode_choice_logsum_fru - trips.mode_choice_logsum_asim) / trips.mode_choice_logsum_fru).hist(bins=50)

Logsums agree


# Run MC validation

see logit._each_nest parent_nest_scale for leaf node, and simulate.compute_nested_utilities for discussion of scales
of alternatives

easiest way is to run a simulation

In [11]:
from fru_utils import get_stuff
t, ru, neu, nu, ns, nested_probs, base_probs = get_stuff(do_these_purposes=["social"])

register joint_tour_participants: no rows with household_id in [982875].
estimation bundle trip_mode_choice not in settings file estimation.yaml


Running with %d trips 482
trip_mode_choice tour_type 'social' (19 trips)
Done


In [12]:
# nested_probs[0] - these are relative in each nest, so the full probs are potentially just the product of these
base_probs[0]

,DRIVEALONEFREE,DRIVEALONEPAY,SHARED2FREE,SHARED2PAY,SHARED3FREE,SHARED3PAY,WALK,BIKE,WALK_LOC,WALK_LRF,WALK_EXP,WALK_HVY,WALK_COM,DRIVE_LOC,DRIVE_LRF,DRIVE_EXP,DRIVE_HVY,DRIVE_COM,TAXI,TNC_SINGLE,TNC_SHARED
trip_id,,,,,,,,,,,,,,,,,,,,,
8805121,0.0,0.0,0.000000e+00,0.0,0.000000e+00,0.0,1.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.266799e-08,7.843494e-08,2.071998e-08
8805125,0.0,0.0,0.000000e+00,0.0,0.000000e+00,0.0,1.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.057327e-08,7.325707e-08,1.950799e-08
8805126,0.0,0.0,0.000000e+00,0.0,0.000000e+00,0.0,1.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.864169e-08,9.376443e-08,2.428075e-08
468119921,0.0,0.0,0.000000e+00,0.0,0.000000e+00,0.0,1.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.096638e-08,2.754998e-10,4.051502e-08
468119922,0.0,0.0,0.000000e+00,0.0,0.000000e+00,0.0,0.999874,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.658716e-05,6.599176e-07,9.832967e-05
468119925,0.0,0.0,0.000000e+00,0.0,0.000000e+00,0.0,0.999875,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.651144e-05,6.601311e-07,9.831370e-05
468119926,0.0,0.0,0.000000e+00,0.0,0.000000e+00,0.0,1.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.568041e-08,2.408144e-10,3.615344e-08
468119927,0.0,0.0,0.000000e+00,0.0,0.000000e+00,0.0,1.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.308505e-08,1.315547e-10,2.041405e-08
468119928,0.0,0.0,0.000000e+00,0.0,0.000000e+00,0.0,1.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.367465e-09,1.493582e-10,1.939597e-08


In [13]:
# for later: this seems to be a non-driving segment - grab workers or something later?
# for now: implement a run with seeding so I can reproduce these counts

# actually, can I manipulate pipeline OR inject table dynamically? If I reproduce a single observation many times we
# can check mode shares
# 1) check tests
# 2) check code in mode choice, maybe duplicate here, see old code below

In [70]:
pipeline._PIPELINE.rng().get_channel_for_df(trips_merged).row_states.index.values

array([   8684833,    8684837,    8685009,    8685013,    8753057,
          8753061,    8753233,    8753237,    8805121,    8805125,
          8805126,    9094329,    9094333,    9717265,    9717269,
         35302033,   35302037,   36757193,   36757197,   36757257,
         36757261,   86627409,   86627413,   86627625,   86627629,
         86673657,   86673658,   86673659,   86673660,   86673661,
        106170305,  106170309,  106289041,  106289045,  106289089,
        106289090,  106289091,  106289092,  106289093,  106289369,
        106289373,  106570377,  106570381,  106570382,  106570383,
        106570465,  106570469,  106741681,  106741682,  106741685,
        106741686,  106741687,  106741688,  106741961,  106741965,
        106804593,  106804597,  106804598,  106804657,  106804661,
        106804662,  106804663,  106804985,  106804989,  126610345,
        126610349,  126610561,  126610565,  126610566,  126610673,
        126610677,  126610873,  126610877,  126628233,  126628

In [ ]:
pipeline._PIPELINE.rng().add_channel("trips", )

In [59]:
pipeline._PIPELINE.rng().Sim

In [65]:
def mode_choice_for_trip(choose_individual_max_utility, trip_id_to_check, num_samples):
    """open pipeline and load stuff for mode choice dev assuming model has been run and pipeline.h5 exists"""
    resume_after = "trip_scheduling"
    model_name = "trip_mode_choice"
    chunk_size = 0  # test_mtc means no chunking

    pipeline.open_pipeline(resume_after)
    # preload any bulky injectables (e.g. skims) not in pipeline
    inject.get_injectable('preload_injectables', None)
    pipeline._PIPELINE.rng().begin_step(model_name)
    #step_name = model_name
    args = {}
    #checkpoint = pipeline.intermediate_checkpoint(model_name)
    inject.set_step_args(args)

    trips = inject.get_table('trips')
    tours_merged = inject.get_table('tours_merged')
    network_los = inject.get_injectable('network_los')

    trace_label = 'trip_mode_choice'
    model_settings_file_name = 'trip_mode_choice.yaml'
    model_settings = config.read_model_settings(model_settings_file_name)

    logsum_column_name = model_settings.get('MODE_CHOICE_LOGSUM_COLUMN_NAME')
    mode_column_name = 'trip_mode'

    trips_df = trips.to_frame()
    print("Running with %d trips", trips_df.shape[0])

    tours_merged = tours_merged.to_frame()
    tours_merged = tours_merged[model_settings['TOURS_MERGED_CHOOSER_COLUMNS']]

    # - trips_merged - merge trips and tours_merged
    trips_merged = pd.merge(
        trips_df,
        tours_merged,
        left_on='tour_id',
        right_index=True,
        how="left")
    assert trips_merged.index.equals(trips.index)

    # setup skim keys
    assert ('trip_period' not in trips_merged)
    trips_merged['trip_period'] = network_los.skim_time_period_label(trips_merged.depart)

    orig_col = 'origin'
    dest_col = 'destination'

    constants = {}
    constants.update(config.get_model_constants(model_settings))
    constants.update({
        'ORIGIN': orig_col,
        'DESTINATION': dest_col
    })

    skim_dict = network_los.get_default_skim_dict()

    odt_skim_stack_wrapper = skim_dict.wrap_3d(orig_key=orig_col, dest_key=dest_col,
                                               dim3_key='trip_period')
    dot_skim_stack_wrapper = skim_dict.wrap_3d(orig_key=dest_col, dest_key=orig_col,
                                               dim3_key='trip_period')
    od_skim_wrapper = skim_dict.wrap('origin', 'destination')

    skims = {
        "odt_skims": odt_skim_stack_wrapper,
        "dot_skims": dot_skim_stack_wrapper,
        "od_skims": od_skim_wrapper,
    }

    model_spec = simulate.read_model_spec(file_name=model_settings['SPEC'])
    nest_specs = config.get_logit_model_settings(model_settings)

    estimator = estimation.manager.begin_estimation('trip_mode_choice')

    choices_list = []


    # grab one, duplicate num_samples times
    trips_segment = trips_merged.loc[trips_merged.index == trip_id_to_check].copy()
    primary_purpose = trips_segment['primary_purpose'].values[0]
    trips_segment = trips_segment.loc[trips_segment.index.repeat(num_samples)]

    # need to add new row_states for rng here, need to ensure there are no collisions with existing keys
    existing_indexes = pipeline._PIPELINE.rng().get_channel_for_df(trips_merged).row_states.index.values
    num_new_indexes = trips_segment.shape[0]
    now find indexes - random draw and then remove existing and do again? simple loop for each, or batch and do
    recursively? simple loop is fast enough here


    trips_segment.index += np.arange(num_samples)
    # name index so tracing knows how to slice
    assert trips_segment.index.name == 'trip_id'


    # #logger.warning("Change seeding back when done with testing")
    # pipeline._PIPELINE.rng.row_states = pd.DataFrame(columns=['row_seed', 'offset'], index=trips_segment.index)
    # pipeline._PIPELINE.rng.row_states["row_seed"] = trips_segment.index.values
    # pipeline._PIPELINE.rng.row_states["offset"] = 0
    pipeline._PIPELINE.rng().add_channel("trips", trips_segment)


    #for primary_purpose, trips_segment in trips_merged.groupby('primary_purpose'):
    #if (do_these_purposes is not None) and (primary_purpose not in do_these_purposes):
    #    continue

    print("trip_mode_choice tour_type '%s' (%s trips)" %
          (primary_purpose, len(trips_segment.index), ))

    coefficients = simulate.get_segment_coefficients(model_settings, primary_purpose)

    locals_dict = {}
    locals_dict.update(constants)
    locals_dict.update(coefficients)

    segment_trace_label = tracing.extend_trace_label(trace_label, primary_purpose)

    expressions.annotate_preprocessors(
        trips_segment, locals_dict, skims,
        model_settings, segment_trace_label)

    locals_dict.update(skims)

    ################ Replace wrapper function
    #     choices = mode_choice_simulate(...)
    spec=simulate.eval_coefficients(model_spec, coefficients, estimator)
    nest_spec = simulate.eval_nest_coefficients(nest_specs, coefficients, segment_trace_label)
    choices = simulate.simple_simulate(
        choosers=trips_segment,
        spec=spec,
        nest_spec=nest_spec,
        skims=skims,
        locals_d=locals_dict,
        chunk_size=chunk_size,
        want_logsums=logsum_column_name is not None,
        trace_label=segment_trace_label,
        trace_choice_name='trip_mode_choice',
        estimator=estimator,
        trace_column_names=None,
        choose_individual_max_utility=choose_individual_max_utility)
    # for consistency, always return dataframe, whether or not logsums were requested
    if isinstance(choices, pd.Series):
        choices = choices.to_frame('choice')
    choices.rename(columns={'logsum': logsum_column_name,
                            'choice': mode_column_name},
                   inplace=True)
    if not choose_individual_max_utility:
        alts = spec.columns
        choices[mode_column_name] = choices[mode_column_name].map(dict(list(zip(list(range(len(alts))), alts))))
    ################
    choices_list.append(choices)
    choices_df_asim = pd.concat(choices_list)

    # update trips table with choices (and potionally logssums)
    trips_df = trips.to_frame()

    #if (do_these_purposes is not None):
    #    trips_df  = trips_df.loc[trips_df.primary_purpose.isin(do_these_purposes)]

    assign_in_place(trips_df, choices_df_asim)
    assert not trips_df[mode_column_name].isnull().any()

    finalise = True
    if finalise:
        inject.set_step_args(None)
        #
        pipeline._PIPELINE.rng().end_step(model_name)
        pipeline.add_checkpoint(model_name)
        if not pipeline.intermediate_checkpoint():
            pipeline.add_checkpoint(pipeline.FINAL_CHECKPOINT_NAME)

        pipeline.close_pipeline()

    print("Done")
    return trips_df

In [66]:
#pipeline.close_pipeline()

In [67]:
choose_individual_max_utility = False
num_samples = 100
trip_id_to_check = 642446345
t_ = mode_choice_for_trip(choose_individual_max_utility=choose_individual_max_utility, trip_id_to_check=trip_id_to_check, num_samples=num_samples)

register joint_tour_participants: no rows with household_id in [982875].
estimation bundle trip_mode_choice not in settings file estimation.yaml


Running with %d trips 482


AssertionError: 

In [18]:
#pipeline.registered_tables()
#list(map(lambda x: x['checkpoint_name'], pipeline._PIPELINE.checkpoints))

In [24]:
num_samples = 100
trip_id_to_check = 642446345

trips_dup = trips_merged.loc[trips_merged.index == trip_id_to_check].copy()
trips_dup = trips_dup.loc[trips_dup.index.repeat(num_samples)]
trips_dup.index += np.arange(num_samples)

# OLD

In [454]:
def simple_simulate_probabilities(trips_segment, spec, nest_spec, locals_d, estimator, tr_label, log_alt_losers, trace_column_names):
    trace_label = tracing.extend_trace_label(tr_label, 'eval_nl')
    logit.validate_nest_spec(nest_spec, trace_label)
    raw_utilities = simulate.eval_utilities(spec, trips_segment, locals_d,
                                            log_alt_losers=log_alt_losers,
                                            trace_label=trace_label, have_trace_targets=False,
                                            estimator=estimator, trace_column_names=trace_column_names)
    nested_exp_utilities = simulate.compute_nested_exp_utilities(raw_utilities, nest_spec)
    nested_probabilities = simulate.compute_nested_probabilities(nested_exp_utilities, nest_spec,
                                                                 trace_label=trace_label)
    # global (flattened) leaf probabilities based on relative nest coefficients (in spec order)
    base_probabilities = simulate.compute_base_probabilities(nested_probabilities, nest_spec, spec)
    return base_probabilities
#simple_simulate_probabilities(trips_segment, spec, nest_spec, locals_dict, estimator, tr_label, log_alt_losers, trace_column_names)


## caculate probabilities with Asim methodology, should be correct
def gimme_probabilities(trips_merged, model_settings, constants, skims, model_spec, nest_spec, estimator, logsum_column_name, mode_column_name, 
          trace_label=None, log_alt_losers=None, trace_column_names=None):
    full_probs = []  # analytical probs

    for primary_purpose, trips_segment in trips_merged.groupby('primary_purpose'):
        #print("trip_mode_choice tour_type '%s' (%s trips)" %
        #            (primary_purpose, len(trips_segment.index), ))
        # name index so tracing knows how to slice
        assert trips_segment.index.name == 'trip_id'

        coefficients = simulate.get_segment_coefficients(model_settings, primary_purpose)

        locals_dict = {}
        locals_dict.update(constants)
        locals_dict.update(coefficients)

        segment_trace_label = tracing.extend_trace_label(trace_label, primary_purpose)

        expressions.annotate_preprocessors(
            trips_segment, locals_dict, skims,
            model_settings, segment_trace_label)

        locals_dict.update(skims)

        spec=simulate.eval_coefficients(model_spec, coefficients, estimator)
        nest_spec = simulate.eval_nest_coefficients(nest_spec, coefficients, segment_trace_label)
        #choices = simple_simulate_rum(trips_segment, spec, nest_spec, locals_dict, estimator, 
        #                              segment_trace_label, log_alt_losers=log_alt_losers, 
        #                              trace_column_names=trace_column_names, seed=seed)
        probs = simple_simulate_probabilities(trips_segment, spec, nest_spec, locals_dict, 
                                              estimator, segment_trace_label, log_alt_losers, 
                                              trace_column_names)
        full_probs.append(probs)
    probs_df = pd.concat(full_probs)
    return probs_df


In [455]:
print(f"{datetime.now()} Start")
pipeline.open_pipeline(resume_after)
inject.get_injectable('preload_injectables', None)
model_name = "trip_mode_choice"
pipeline._PIPELINE.rng().begin_step(model_name)

step_name = model_name
args = {}
checkpoint = pipeline.intermediate_checkpoint(model_name)
inject.set_step_args(args)

trips = inject.get_table('trips')
tours_merged = inject.get_table('tours_merged')
network_los = inject.get_injectable('network_los')
chunk_size = 0

trace_label = 'trip_mode_choice'
model_settings_file_name = 'trip_mode_choice.yaml'
model_settings = config.read_model_settings(model_settings_file_name)

logsum_column_name = model_settings.get('MODE_CHOICE_LOGSUM_COLUMN_NAME')
mode_column_name = 'trip_mode'
trips_df = trips.to_frame()
#print("Running with %d trips", trips_df.shape[0])
tours_merged = tours_merged.to_frame()
tours_merged = tours_merged[model_settings['TOURS_MERGED_CHOOSER_COLUMNS']]
# - trips_merged - merge trips and tours_merged
trips_merged = pd.merge(
    trips_df,
    tours_merged,
    left_on='tour_id',
    right_index=True,
    how="left")
assert trips_merged.index.equals(trips.index)

# setup skim keys
assert ('trip_period' not in trips_merged)
trips_merged['trip_period'] = network_los.skim_time_period_label(trips_merged.depart)

orig_col = 'origin'
dest_col = 'destination'

constants = {}
constants.update(config.get_model_constants(model_settings))
constants.update({
    'ORIGIN': orig_col,
    'DESTINATION': dest_col
})

skim_dict = network_los.get_default_skim_dict()

odt_skim_stack_wrapper = skim_dict.wrap_3d(orig_key=orig_col, dest_key=dest_col,
                                           dim3_key='trip_period')
dot_skim_stack_wrapper = skim_dict.wrap_3d(orig_key=dest_col, dest_key=orig_col,
                                           dim3_key='trip_period')
od_skim_wrapper = skim_dict.wrap('origin', 'destination')

skims = {
    "odt_skims": odt_skim_stack_wrapper,
    "dot_skims": dot_skim_stack_wrapper,
    "od_skims": od_skim_wrapper,
}

model_spec = simulate.read_model_spec(file_name=model_settings['SPEC'])
nest_spec = config.get_logit_model_settings(model_settings)

estimator = estimation.manager.begin_estimation('trip_mode_choice')


all_choices = []
for i in range(100):
    if i % 10 == 0:
        print(f"{datetime.now()} iteration {i}")
    choices_df = stuff(trips_merged, model_settings, constants, skims, model_spec, nest_spec, estimator, logsum_column_name, mode_column_name, 
              trace_label=trace_label, log_alt_losers=None, trace_column_names=None, seed=None)
    all_choices.append(choices_df)
all_choices = pd.concat(all_choices, axis=1)

probs_nl = gimme_probabilities(trips_merged, model_settings, constants, skims, model_spec, nest_spec,
                               estimator, logsum_column_name, mode_column_name,trace_label=trace_label, 
                               log_alt_losers=None, trace_column_names=None)

# update trips table with choices (and potionally logssums)
#trips_df = trips.to_frame()
#
#assign_in_place(trips_df, choices_df)
#assert not trips_df[mode_column_name].isnull().any()


finalise = True
if finalise:
    inject.set_step_args(None)
    #
    pipeline._PIPELINE.rng().end_step(model_name)
    pipeline.add_checkpoint(model_name)
    if not pipeline.intermediate_checkpoint():
        pipeline.add_checkpoint(pipeline.FINAL_CHECKPOINT_NAME)

    pipeline.close_pipeline()

print(f"{datetime.now()} End")

register joint_tour_participants: no rows with household_id in [982875].
estimation bundle trip_mode_choice not in settings file estimation.yaml


2021-09-01 14:59:46.353007 Start
2021-09-01 14:59:52.976736 End


In [463]:
#all_choices.merge(choices_df_asim[['trip_mode']].rename(columns={'trip_mode': 'asim'}), left_index=True, right_index=True)
val_counts = all_choices.apply(lambda x: x.value_counts(), axis=1).fillna(0)
val_counts = val_counts / all_choices.shape[1]
#val_counts = val_counts.merge(choices_df_asim[['trip_mode']].rename(columns={'trip_mode': 'asim'}), left_index=True, right_index=True)
#val_counts['prob_of_asim_choice'] = val_counts.apply(lambda x: x[x.asim], axis=1)  # this is what our simulation says w.r.t. to asim choice
# for 100% and many samples should mostly agree

In [464]:
#val_counts['prob_of_asim_choice'].hist(bins=100);

In [490]:
mode_share_analytical = (probs_nl.sum(axis=0) / probs_nl.shape[0]).to_frame('analytical')
assert np.allclose(mode_share_analytical.sum(), 1)
mode_share_rum = (val_counts.sum(axis=0) / val_counts.shape[0]).to_frame('experiment')
assert np.allclose(mode_share_rum.sum(), 1)
full_share = mode_share_analytical.join(mode_share_rum, how='outer').fillna(0)
with pd.option_context("precision", 3):
    display((100.0 * full_share).T)

,BIKE,DRIVEALONEFREE,DRIVEALONEPAY,DRIVE_COM,DRIVE_EXP,DRIVE_HVY,DRIVE_LOC,DRIVE_LRF,SHARED2FREE,SHARED2PAY,SHARED3FREE,SHARED3PAY,TAXI,TNC_SHARED,TNC_SINGLE,WALK,WALK_COM,WALK_EXP,WALK_HVY,WALK_LOC,WALK_LRF
analytical,3.152,0.852,0.0,0.0,0.0,0.0,0.0,0.0,0.685,0.0,0.129,0.0,0.182,0.260,1.334,63.708,0.0,0.0,0.455,18.355,10.887
experiment,3.243,0.880,0.0,0.0,0.0,0.0,0.0,0.0,0.772,0.0,0.156,0.0,0.174,0.259,1.434,63.243,0.0,0.0,0.394,18.639,10.807


In [631]:
num_samples = 10

rng_test = default_rng(23423)

print(f"{datetime.now()} Start")
pipeline.open_pipeline(resume_after)
inject.get_injectable('preload_injectables', None)
model_name = "trip_mode_choice"
pipeline._PIPELINE.rng().begin_step(model_name)

step_name = model_name
args = {}
checkpoint = pipeline.intermediate_checkpoint(model_name)
inject.set_step_args(args)

trips = inject.get_table('trips')
tours_merged = inject.get_table('tours_merged')
network_los = inject.get_injectable('network_los')
chunk_size = 0

trace_label = 'trip_mode_choice'
model_settings_file_name = 'trip_mode_choice.yaml'
model_settings = config.read_model_settings(model_settings_file_name)

logsum_column_name = model_settings.get('MODE_CHOICE_LOGSUM_COLUMN_NAME')
mode_column_name = 'trip_mode'
trips_df = trips.to_frame()
#print("Running with %d trips", trips_df.shape[0])
tours_merged = tours_merged.to_frame()
tours_merged = tours_merged[model_settings['TOURS_MERGED_CHOOSER_COLUMNS']]
# - trips_merged - merge trips and tours_merged
trips_merged = pd.merge(
    trips_df,
    tours_merged,
    left_on='tour_id',
    right_index=True,
    how="left")
assert trips_merged.index.equals(trips.index)

# setup skim keys
assert ('trip_period' not in trips_merged)
trips_merged['trip_period'] = network_los.skim_time_period_label(trips_merged.depart)

orig_col = 'origin'
dest_col = 'destination'

constants = {}
constants.update(config.get_model_constants(model_settings))
constants.update({
    'ORIGIN': orig_col,
    'DESTINATION': dest_col
})

skim_dict = network_los.get_default_skim_dict()
odt_skim_stack_wrapper = skim_dict.wrap_3d(orig_key=orig_col, dest_key=dest_col,
                                           dim3_key='trip_period')
dot_skim_stack_wrapper = skim_dict.wrap_3d(orig_key=dest_col, dest_key=orig_col,
                                           dim3_key='trip_period')
od_skim_wrapper = skim_dict.wrap('origin', 'destination')
skims = {
    "odt_skims": odt_skim_stack_wrapper,
    "dot_skims": dot_skim_stack_wrapper,
    "od_skims": od_skim_wrapper,
}
model_spec = simulate.read_model_spec(file_name=model_settings['SPEC'])
nest_spec = config.get_logit_model_settings(model_settings)
estimator = estimation.manager.begin_estimation('trip_mode_choice')

all_choices_zenith = []
all_choices = []
for i in range(num_samples):
    
    seed = rng_test.integers(0, 100000) #int(9.3 * (i+1)**3)  # why not
    
    if i % 50 == 0:
        print(f"{datetime.now()} iteration {i}")
    choices_df_zenith = stuff_zenith(trips_merged, model_settings, constants, skims, model_spec, nest_spec, estimator, logsum_column_name, mode_column_name, 
              trace_label=trace_label, log_alt_losers=None, trace_column_names=None, seed=seed)
    all_choices_zenith.append(choices_df_zenith)

    choices_df = stuff(trips_merged, model_settings, constants, skims, model_spec, nest_spec, estimator, logsum_column_name, mode_column_name, 
          trace_label=trace_label, log_alt_losers=None, trace_column_names=None, seed=seed)
    all_choices.append(choices_df)
    
    t_ = choices_df_zenith.merge(choices_df, left_index=True, right_index=True, suffixes=['_zenith', '_asim'])
    diffs = t_.loc[t_.trip_mode_zenith != t_.trip_mode_asim]
    
    #print(f"seed {seed} leads to {diffs.shape[0]} differences. tripids {diffs.index}")

all_choices_zenith = pd.concat(all_choices_zenith, axis=1)
all_choices = pd.concat(all_choices, axis=1)

probs_nl = gimme_probabilities(trips_merged, model_settings, constants, skims, model_spec, nest_spec,
                               estimator, logsum_column_name, mode_column_name,trace_label=trace_label, 
                               log_alt_losers=None, trace_column_names=None)

finalise = True
if finalise:
    inject.set_step_args(None)
    #
    pipeline._PIPELINE.rng().end_step(model_name)
    pipeline.add_checkpoint(model_name)
    if not pipeline.intermediate_checkpoint():
        pipeline.add_checkpoint(pipeline.FINAL_CHECKPOINT_NAME)

    pipeline.close_pipeline()

print(f"{datetime.now()} End")

In [643]:
print(f"Have {all_choices_zenith.shape[1]} samples")

Have 2038 samples


In [633]:
val_counts_zenith = all_choices_zenith.apply(lambda x: x.value_counts(), axis=1).fillna(0)
val_counts_zenith = val_counts_zenith / all_choices_zenith.shape[1]

val_counts = all_choices.apply(lambda x: x.value_counts(), axis=1).fillna(0)
val_counts = val_counts / all_choices.shape[1]

In [634]:
mode_share_rum_zenith = (val_counts_zenith.sum(axis=0) / val_counts_zenith.shape[0]).to_frame('experiment_zenith')
assert np.allclose(mode_share_rum_zenith.sum(), 1)
full_share_incl_zenith = full_share.merge(mode_share_rum_zenith, left_index=True, right_index=True, how='outer').fillna(0)
with pd.option_context("precision", 3):
    display((100.0 * full_share_incl_zenith).T)

,BIKE,DRIVEALONEFREE,DRIVEALONEPAY,DRIVE_COM,DRIVE_EXP,DRIVE_HVY,DRIVE_LOC,DRIVE_LRF,SHARED2FREE,SHARED2PAY,SHARED3FREE,SHARED3PAY,TAXI,TNC_SHARED,TNC_SINGLE,WALK,WALK_COM,WALK_EXP,WALK_HVY,WALK_LOC,WALK_LRF
analytical,3.152,0.852,0.0,0.0,0.0,0.0,0.0,0.0,0.685,0.0,0.129,0.0,0.182,0.260,1.334,63.708,0.0,0.0,0.455,18.355,10.887
experiment,3.243,0.880,0.0,0.0,0.0,0.0,0.0,0.0,0.772,0.0,0.156,0.0,0.174,0.259,1.434,63.243,0.0,0.0,0.394,18.639,10.807
experiment_zenith,3.150,0.874,0.0,0.0,0.0,0.0,0.0,0.0,0.680,0.0,0.136,0.0,0.154,0.236,1.285,63.767,0.0,0.0,0.515,18.824,10.379
